In [5]:
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess

In [2]:
df = pd.read_csv("/kaggle/input/text-classification-documentation/df_file.csv")
df.head()

,Text,Label
0,Budget to set scene for election\n \n Gordon B...,0
1,Army chiefs in regiments decision\n \n Militar...,0
2,Howard denies split over ID cards\n \n Michael...,0
3,Observers to monitor UK election\n \n Minister...,0
4,Kilroy names election seat target\n \n Ex-chat...,0


In [3]:
df["Label"].value_counts()

Label
1    511
4    510
0    417
2    401
3    386
Name: count, dtype: int64

In [6]:
df["preprocessed_Text"] = df["Text"].apply(simple_preprocess)
df.head()

,Text,Label,preprocessed_Text
0,Budget to set scene for election\n \n Gordon B...,0,"[budget, to, set, scene, for, election, gordon..."
1,Army chiefs in regiments decision\n \n Militar...,0,"[army, chiefs, in, regiments, decision, milita..."
2,Howard denies split over ID cards\n \n Michael...,0,"[howard, denies, split, over, id, cards, micha..."
3,Observers to monitor UK election\n \n Minister...,0,"[observers, to, monitor, uk, election, ministe..."
4,Kilroy names election seat target\n \n Ex-chat...,0,"[kilroy, names, election, seat, target, ex, ch..."


In [7]:
from gensim.models import Word2Vec

model = Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [8]:
model.build_vocab(df.preprocessed_Text)

In [9]:
model.train(df.preprocessed_Text, total_examples=model.corpus_count, epochs=model.epochs)

(3218434, 4111210)

In [10]:
model.wv.most_similar("bad")

[('fun', 0.9799472689628601),
 ('sense', 0.9483348727226257),
 ('somebody', 0.947866678237915),
 ('actually', 0.9477484226226807),
 ('oh', 0.9454336166381836),
 ('pretty', 0.9452409744262695),
 ('extremely', 0.9448727369308472),
 ('obviously', 0.9442196488380432),
 ('nice', 0.9408380389213562),
 ('everything', 0.9396749138832092)]

In [11]:
l = model.wv
len(l) # num of words

18588

In [12]:
model.vector_size

100

In [17]:
def embed(preprocessed_text):
    embedding_words = \
    [
        model.wv[token]
        for token in preprocessed_text if token in model.wv
    ]
    
    if embedding_words:
        return np.mean(embedding_words, axis=0)
    else:
        return np.zeros(model.vector_size)

In [18]:
df["embedding_text"] = df["preprocessed_Text"].apply(embed)
df.head()

,Text,Label,preprocessed_Text,embedding_text
0,Budget to set scene for election\n \n Gordon B...,0,"[budget, to, set, scene, for, election, gordon...","[-0.49979162, -0.016332231, -0.15140297, 0.243..."
1,Army chiefs in regiments decision\n \n Militar...,0,"[army, chiefs, in, regiments, decision, milita...","[-0.4292663, 0.10876476, -0.09699733, 0.144302..."
2,Howard denies split over ID cards\n \n Michael...,0,"[howard, denies, split, over, id, cards, micha...","[-0.23822846, -0.00672493, -0.07900661, -0.006..."
3,Observers to monitor UK election\n \n Minister...,0,"[observers, to, monitor, uk, election, ministe...","[-0.4751318, 0.08111424, -0.12235079, 0.178975..."
4,Kilroy names election seat target\n \n Ex-chat...,0,"[kilroy, names, election, seat, target, ex, ch...","[-0.06479568, -0.361393, -0.022267913, 0.07522..."


In [24]:
X = np.stack(df["embedding_text"])
y = df["Label"]

In [33]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
X = sc.fit_transform(X)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [36]:
from sklearn.naive_bayes import MultinomialNB

cls = MultinomialNB()
cls.fit(X_train, y_train)

MultinomialNB()

In [41]:
from sklearn.metrics import accuracy_score

y_pred = cls.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 74.83%


In [42]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 84.94%


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
